In [1]:
# ==========================================================
#                       LIBRARIES
# ==========================================================

import pandas as pd
import numpy as np

from plotly import graph_objects as go
from plotly.subplots import make_subplots


In [28]:
# load data

gc = pd.read_csv('x_y_z_bin_gc.csv').iloc[:, 1:]
com = pd.DataFrame(np.load('x_y_z_dist_com_dist_rm.npy'), columns=['x', 'y', 'z', 'dist_com', 'dist_rm'])
sasa = pd.DataFrame(np.load('x_y_z_dist_surf.npy'), columns=['x', 'y', 'z', 'dist_surf'])


In [32]:
data_combined = pd.concat([gc, com[['dist_com', 'dist_rm']], sasa['dist_surf']], axis=1)

data_combined


,x,y,z,bin,gc_content_percentage,dist_com,dist_rm,dist_surf
0,-0.3264,0.3988,-0.2650,0,33.65,0.581433,0.238979,1.652275
1,-0.3026,0.4367,-0.2681,500000,43.05,0.596533,0.200534,1.616847
2,-0.2171,0.5026,-0.2616,1000000,60.50,0.606912,0.119064,1.520853
3,-0.0999,0.6025,-0.2160,1500000,54.07,0.645951,0.121146,1.366583
4,0.0170,0.6524,-0.1841,2000000,58.55,0.674883,0.223344,1.248294
...,...,...,...,...,...,...,...,...
446,-0.3537,0.2988,0.4387,240500000,39.82,0.635022,0.139890,1.412877
447,-0.3457,0.2375,0.4932,241000000,39.81,0.644606,0.141894,1.407866
448,-0.3446,0.1994,0.5161,241500000,40.06,0.649107,0.150746,1.414836
449,-0.3101,0.2030,0.5102,242000000,40.85,0.627557,0.145351,1.389543


In [30]:
fig = go.Figure(data=go.Scatter3d(
    x=sasa['x'],
    y=sasa['y'],
    z=sasa['z'],
    mode='lines',
    line=dict(
        width=5
    )
))

fig.show()
